# Weak References

¿Para qué sirven las referencias débiles?
Las referencias débiles son útiles en situaciones donde quieres mantener una referencia a un objeto, pero no quieres que esa referencia impida que el objeto sea eliminado cuando ya no sea necesario. Algunos casos de uso comunes incluyen:

Caches: Si estás implementando un caché, puedes usar referencias débiles para almacenar los objetos en el caché. Si el objeto ya no está en uso en otras partes del programa, puede ser recolectado, liberando memoria.

Estructuras de datos que no deben interferir con la gestión de memoria: Por ejemplo, si tienes una estructura de datos que mantiene referencias a objetos, pero no quieres que esa estructura impida que los objetos sean eliminados cuando ya no se usen.

Evitar ciclos de referencias: En algunos casos, las referencias débiles pueden ayudar a evitar ciclos de referencias que podrían causar fugas de memoria.

In [14]:
import gc, weakref, sys

In [20]:
class Cat:

    def __init__(self, name) -> None:
        self.name = name

    def __del__(self):
        print(f"bye")

cat = Cat("michi")

In [23]:
weak_obj = weakref.ref(cat)
weak_obj().name

'michi'

In [24]:
del cat
weak_obj()

bye


In [13]:
# Initial rc
initial_rc = sys.getrefcount(cat)
print(f"Initial rc: {initial_rc}")

# Creating weak reference
def speak(message):
    print(f"{message}")

cat_weak_ref = weakref.ref(cat, speak)
print(f"rc after weakref.ref: {sys.getrefcount(cat)}")

# Creando un proxy usando weakref.proxy
proxy_cat = weakref.proxy(cat)
print(f"rc after the weakref.proxy: {sys.getrefcount(cat)}")

Initial rc: 2
rc after weakref.ref: 2
rc after the weakref.proxy: 2


In [9]:
del cat
print(sys.getrefcount(cat))

bye
<weakref at 0x00000198744DDBC0; dead>


NameError: name 'cat' is not defined

In [10]:
print(cat_weak_ref())

None


In [26]:
sys.getrefcount(cat), sys.getrefcount(cat_weak_ref)

(2, 2)

In [27]:
gc.get_referents(cat), gc.is_finalized(cat), gc.callbacks

(['michi', __main__.Cat],
 False,
 [<bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x000001770EB6F740>>])

In [28]:
gc.collect(generation=0)

46

# Proxy Objects

Allow direct attribute access and method calls without requiring the calling of a reference as with weakref.ref().

In [ ]:
class Dog:

    def __init__(self, name) -> None:
        self.name = name

    def bark(self):
        print("Wauf!")

dog = Dog("bartolomeo")
proxy_obj = weakref.proxy(dog)

# Access methods and attributes directly via the proxy
proxy_obj.bark()

In [ ]:
sys.getrefcount(cat)

# Cyclical References

In [8]:
class C:
    def __del__(self):
        print(f"bye")

c = C()
c.c = c

print(c.c.c.c)

In [9]:
gc.collect()

52

In [12]:
# Avoiding circular references
c = C()
c.x = 1
c.c = weakref.proxy(c)
print(c.c.c.c.c.c.c.c.x)

1


In [13]:
# C will be garbage collected since has no strong references,
c = None

bye


# Caching use case

In [14]:
from weakref import WeakValueDictionary, WeakKeyDictionary
d = WeakKeyDictionary()
a = "a"
d[a] = 1
d[a]

TypeError: cannot create weak reference to 'str' object

In [16]:
class C:
    def __del__(self):
        print(f"bye")

d = WeakKeyDictionary()
c = C()
d[c] = 5
d[c]

bye


5

In [18]:
print(list(d.items()))

[(<__main__.C object at 0x000001770EE8AB70>, 5)]


In [19]:
# This dictionary does not keep the objects alive, so it's useful to avoid
# memory leaks with memory expensive values on caching
c = None
print(list(d.items()))

bye
[]
